**Author**: Siddhant Sutar

Import libraries.

In [90]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix

Import training and test samples.

In [91]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

Function to split into tokens.

In [92]:
def tokens(x):
    return x.split(', ')

Initialize vectorizer.

In [93]:
vectorizer = TfidfVectorizer(tokenizer=tokens, lowercase=False, use_idf=False, norm=False, binary=True)

Create feature vectors for actors.

In [94]:
actors = vectorizer.fit_transform(train["Cast"].fillna(''))
actors_list = vectorizer.get_feature_names()[1:]
actors_list = [x.encode('UTF8') for x in actors_list]

Create feature vectors for directors.

In [96]:
directors = vectorizer.fit_transform(train["Director"].fillna(''))
directors_list = vectorizer.get_feature_names()[1:]
directors_list = [x.encode('UTF8') for x in directors_list]

Create feature vectors for genres.

In [97]:
genres = vectorizer.fit_transform(train["Genre"].fillna(''))
genres_list = [str(each) for each in vectorizer.get_feature_names()[1:]]

Create feature vectors for plot keywords.

In [98]:
stopset = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(use_idf=False, norm=False, strip_accents='ascii', stop_words=stopset, binary=True)
plot = vectorizer.fit_transform(train["FullPlot"].fillna(''))
keywords_list = vectorizer.get_feature_names()

Predictor features.

In [111]:
features = []
g = ["Crime", "Drama", "Thriller"]
d = ["Alejandro González Iñárritu"]
a = ["Sean Penn", "Naomi Watts", "Danny Huston"]
keywords = [] #["corruption", "vampire", "girl"]
for each in g:
    features.append(genres[:, genres_list.index(str(each))+1].toarray().flatten())
for each in d:
    features.append(directors[:, directors_list.index(str(each))+1].toarray().flatten())
for each in a:
    features.append(actors[:, actors_list.index(str(each))+1].toarray().flatten())

Initialize and fit.

In [112]:
X = csr_matrix(features).transpose()
y = train.imdbRating
glm = LinearRegression()
glm.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Predict the outcome rating.

In [113]:
glm.predict(np.ones(len(features)))[0]

8.1484796075118364